In [173]:
import numpy as np
import pandas as pd
from scipy.sparse.linalg import svds

In [174]:
book_df = pd.read_csv('Books.csv')
ratings_df = pd.read_csv('Ratings.csv').sample(60000)
user_df = pd.read_csv('Users.csv')

<ipython-input-174-1407295537a8>:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  book_df = pd.read_csv('Books.csv')


In [175]:
book_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271360 entries, 0 to 271359
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   ISBN                 271360 non-null  object
 1   Book-Title           271360 non-null  object
 2   Book-Author          271358 non-null  object
 3   Year-Of-Publication  271360 non-null  object
 4   Publisher            271358 non-null  object
 5   Image-URL-S          271360 non-null  object
 6   Image-URL-M          271360 non-null  object
 7   Image-URL-L          271357 non-null  object
dtypes: object(8)
memory usage: 16.6+ MB


In [176]:
ratings_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 60000 entries, 231618 to 287364
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   User-ID      60000 non-null  int64 
 1   ISBN         60000 non-null  object
 2   Book-Rating  60000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.8+ MB


In [177]:
user_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278858 entries, 0 to 278857
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   User-ID   278858 non-null  int64  
 1   Location  278858 non-null  object 
 2   Age       168096 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 6.4+ MB


In [178]:
ratings_df.isnull().sum()


,0
User-ID,0
ISBN,0
Book-Rating,0


In [179]:
ratings_df = ratings_df[ratings_df['Book-Rating'] != 0]

In [180]:
book_df.isnull().sum()

,0
ISBN,0
Book-Title,0
Book-Author,2
Year-Of-Publication,0
Publisher,2
Image-URL-S,0
Image-URL-M,0
Image-URL-L,3


In [181]:
book_df.dropna(how='any', inplace=True)

In [182]:
user_df.isnull().sum()

,0
User-ID,0
Location,0
Age,110762


In [183]:
user_rating_df = ratings_df.merge(user_df, left_on = 'User-ID', right_on = 'User-ID')

In [184]:
ratings_df.head(10)

,User-ID,ISBN,Book-Rating
1092163,261899,0385265700,7
77513,16654,8475776159,7
1099245,264031,0312277032,10
76728,16634,0425053245,8
975177,235105,0345452550,9
480641,114720,0446606189,8
1137968,273979,0786867795,9
272290,63595,0380814803,9
787502,190627,0060956305,7
234585,54374,0449904067,10


In [185]:
user_rating_df.head(10)

,User-ID,ISBN,Book-Rating,Location,Age
0,261899,0385265700,7,"washingtonville, new york, usa",52.0
1,16654,8475776159,7,"málaga, andalucia, spain",35.0
2,264031,0312277032,10,"traverse city, michigan, usa",34.0
3,16634,0425053245,8,"little rock, arkansas, usa",42.0
4,235105,0345452550,9,"st louis, missouri, usa",46.0
5,114720,0446606189,8,"st. louis park, minnesota, usa",NaN
6,273979,0786867795,9,"bloomsburg, pennsylvania, usa",20.0
7,63595,0380814803,9,"chesapeake, virginia,",41.0
8,190627,0060956305,7,"chicago, ,",31.0
9,54374,0449904067,10,"london, ontario, canada",24.0


In [186]:
book_user_rating = book_df.merge(user_rating_df, left_on = 'ISBN', right_on = 'ISBN')
book_user_rating = book_user_rating[['ISBN', 'Book-Title', 'Book-Author', 'User-ID', 'Book-Rating']]
book_user_rating.reset_index(drop=True, inplace=True)
book_user_rating.head(10)

,ISBN,Book-Title,Book-Author,User-ID,Book-Rating
0,0399135782,The Kitchen God's Wife,Amy Tan,238557,10
1,0440234743,The Testament,John Grisham,131703,9
2,0440234743,The Testament,John Grisham,54884,8
3,0440234743,The Testament,John Grisham,128463,7
4,0440234743,The Testament,John Grisham,133706,8
5,0440234743,The Testament,John Grisham,187944,6
6,0440234743,The Testament,John Grisham,244761,6
7,0440234743,The Testament,John Grisham,120396,10
8,0440234743,The Testament,John Grisham,207825,7
9,0440234743,The Testament,John Grisham,31560,6


In [187]:
book_user_rating.size

100500

In [188]:
d ={}
for i,j in enumerate(book_user_rating.ISBN.unique()):
    d[j] =i
book_user_rating['unique_id_book'] = book_user_rating['ISBN'].map(d)
book_user_rating.head(10)

,ISBN,Book-Title,Book-Author,User-ID,Book-Rating,unique_id_book
0,0399135782,The Kitchen God's Wife,Amy Tan,238557,10,0
1,0440234743,The Testament,John Grisham,131703,9,1
2,0440234743,The Testament,John Grisham,54884,8,1
3,0440234743,The Testament,John Grisham,128463,7,1
4,0440234743,The Testament,John Grisham,133706,8,1
5,0440234743,The Testament,John Grisham,187944,6,1
6,0440234743,The Testament,John Grisham,244761,6,1
7,0440234743,The Testament,John Grisham,120396,10,1
8,0440234743,The Testament,John Grisham,207825,7,1
9,0440234743,The Testament,John Grisham,31560,6,1


In [190]:
users_books_pivot_matrix_df = book_user_rating.pivot(index='User-ID',
                                                        columns='unique_id_book',
                                                        values='Book-Rating').fillna(0)
users_books_pivot_matrix_df.head(10)

unique_id_book,0,1,2,3,4,5,6,7,8,9,...,15499,15500,15501,15502,15503,15504,15505,15506,15507,15508
User-ID,,,,,,,,,,,,,,,,,,,,,
17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
56,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
99,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
141,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
165,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
178,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
242,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
243,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
244,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [191]:
users_books_pivot_matrix_df = users_books_pivot_matrix_df.values
users_books_pivot_matrix_df

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

Разложение исходной матрицы, это позволит нам выявить скрытые факторы, влияющие на оценки пользователями книг

In [192]:
NUMBER_OF_FACTORS_MF = 15

#Performs matrix factorization of the original user item matrix
U, sigma, Vt = svds(users_books_pivot_matrix_df, k = NUMBER_OF_FACTORS_MF)

In [193]:
sigma = np.diag(sigma)
sigma.shape

(15, 15)

Двойное матричное умножение для получения предположительных оценок от каждого пользователя к каждой книге

In [194]:
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt)
all_user_predicted_ratings

array([[ 7.47292451e-35,  9.23848450e-22,  9.43227627e-22, ...,
         5.29178246e-35,  4.52847926e-23,  3.90288890e-35],
       [ 1.52125548e-34, -3.79421968e-22, -3.26873968e-22, ...,
        -8.45245253e-36, -1.07288692e-22, -6.25816949e-36],
       [-1.18946873e-34, -9.78462062e-22, -9.24403068e-22, ...,
        -2.65406236e-36, -2.40435509e-24, -5.83230833e-35],
       ...,
       [ 4.77261322e-35, -4.77938466e-23, -1.41401380e-23, ...,
        -1.78723176e-35, -2.77692097e-23, -4.40905400e-37],
       [-4.50350718e-34, -5.08532564e-21, -5.13271821e-21, ...,
        -1.70991687e-34, -2.56303725e-22, -2.31564933e-34],
       [ 3.49495272e-21,  1.29884431e-07,  1.33886750e-07, ...,
        -9.25428022e-23, -6.12001269e-11,  2.99864156e-21]])